In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()
import gzip
tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
rider_otw_raw_file='1582100338-1582100338-CaptainOrderStatus-onTheWay-20200218-0-0.json.gz'

In [5]:
def parse_time(x):
    return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

with gzip.open(rider_otw_raw_file) as f:
    rider_otw_raw=pd.read_json(rider_otw_raw_file,lines=True)

In [6]:
rider_otw_raw.head()
rider_otw_raw.eventProps.iloc[2]
rider_otw_raw.profile.iloc[2]
rider_otw_raw.deviceInfo.iloc[2]

,eventName,ts,eventProps,profile,deviceInfo
0,CaptainOrderStatus-onTheWay,20200218000002,"{'city': 'Ludhiana', 'pickupClusters': 'DMC', ...",{'all_identities': ['5e2b0f65257046721c75b267'...,"{'osVersion': '7.1.1', 'sdkVersion': '30601', ..."
1,CaptainOrderStatus-onTheWay,20200218000053,"{'city': 'Madurai', 'pickupClusters': '', 'dro...",{'all_identities': ['5de4e66dac56a62ec5e544b8'...,"{'osVersion': '8.0.0', 'sdkVersion': '30601', ..."
2,CaptainOrderStatus-onTheWay,20200218000100,"{'city': 'Guntur', 'pickupClusters': 'Arundelp...",{'all_identities': ['5c4f4b78976854537ab13937'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
3,CaptainOrderStatus-onTheWay,20200218000103,"{'city': 'Kolkata', 'pickupClusters': 'Beniapu...",{'all_identities': ['5cc996a94fd5d569a7137323'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
4,CaptainOrderStatus-onTheWay,20200218000111,"{'city': 'Bangalore', 'pickupClusters': 'Basav...",{'all_identities': ['5d676d9e3576981183974005'...,"{'osVersion': '8.1.0', 'sdkVersion': '30601', ..."


{'city': 'Guntur',
 'pickupClusters': 'Arundelpet',
 'dropClusters': '',
 'paymentType': 'cash',
 'orderType': 'app',
 'clientId': 'self',
 'showOtp': 'true',
 'CT Source': 'API',
 'estimateAmount': 0,
 'lat': 16.300024,
 'lng': 80.44376,
 'identity': '5c4f4b78976854537ab13937',
 'serviceType': '5c4317d8487a78564f91f4c7',
 'orderId': '5e4adbd69a773437e4b7a901',
 'timeBucket': '00:00-00:30',
 'orderDate': '2020-02-18',
 'otp': '0976'}

{'all_identities': ['5c4f4b78976854537ab13937',
  'knmrao84@gmail.com',
  '5ca5c0ab54bc7263ff268a7b'],
 'identity': '5c4f4b78976854537ab13937,5ca5c0ab54bc7263ff268a7b',
 'platform': 'Android',
 'phone': 919491195812,
 'name': 'KOTA NAGAMALLESWARARAO',
 'email': 'knmrao84@gmail.com',
 'push_token': 'fcm:coh4PjogKCo:APA91bF1fEK3hB1HPXus9iGtUw_tp5pwaPdk4t5FAkGhM3v3SpBUipdaLZPL1BVVlNCeYdIgRjkOQKtDbKRYz7Cv4_XlW49PNiGcwzKvsRDF7a1z3Uk95aA6TzgTKjYiCameLgZKC7qD'}

{'osVersion': '9',
 'sdkVersion': '30601',
 'make': 'vivo',
 'model': 'Others',
 'appVersion': '3.10.5',
 'browser': 'MobileApp',
 'dpi': 480,
 'dimensions': {'width': 69, 'height': 142, 'unit': 'mm'}}

In [7]:
from datetime import datetime
def parse_rider_otw_rendered(file_name):
    def get_property_value_for_col(df,source_column,target_column,property_key):
        def get_property(payload,property_key):
            try:
                identity_index=[i for i, s in enumerate(payload) if property_key in s][0]
            except:
                return None
            value=list(payload.values())[identity_index]
            return value
        df.loc[:,target_column]=df[source_column].progress_apply(lambda x: get_property(x,property_key))
        return df
    def parse_time(x):
        return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

    with gzip.open(rider_otw_raw_file) as f:
        rider_otw_raw=pd.read_json(rider_otw_raw_file,lines=True)

    rider_otw_raw['time']=rider_otw_raw['ts'].progress_apply(parse_time)
    source_column='eventProps'
    target_column='order_id'
    property_key='orderId'

    rider_otw_raw=get_property_value_for_col(rider_otw_raw,source_column,target_column,property_key)
    source_column='profile'
    target_column='captain_id'
    property_key='identity'
    rider_otw_raw=get_property_value_for_col(rider_otw_raw,source_column,target_column,property_key)
    
    source_column='deviceInfo'
    target_column='app_version'
    property_key='appVersion'
    rider_otw_raw=get_property_value_for_col(rider_otw_raw,source_column,target_column,property_key)

    return rider_otw_raw

rider_otw=parse_rider_otw_rendered(rider_otw_raw_file)


rider_otw.head(2)

100%|██████████| 599455/599455 [00:01<00:00, 321279.84it/s]


,eventName,ts,eventProps,profile,deviceInfo,time,order_id,captain_id,app_version
0,CaptainOrderStatus-onTheWay,20200218000002,"{'city': 'Ludhiana', 'pickupClusters': 'DMC', ...",{'all_identities': ['5e2b0f65257046721c75b267'...,"{'osVersion': '7.1.1', 'sdkVersion': '30601', ...",2020-02-18 00:00:02,5e4ad582865d9017b53de2dd,5e2b0f65257046721c75b267,3.10.5
1,CaptainOrderStatus-onTheWay,20200218000053,"{'city': 'Madurai', 'pickupClusters': '', 'dro...",{'all_identities': ['5de4e66dac56a62ec5e544b8'...,"{'osVersion': '8.0.0', 'sdkVersion': '30601', ...",2020-02-18 00:00:53,5e4adbd89a773437e4b7a902,5de4e66dac56a62ec5e544b8,3.10.5


In [10]:
rider_otw[['eventName','time','order_id','captain_id','app_version']].to_csv('rider_otw_20200218.csv',index=False)